In [3]:
import pickle, pandas as pd, numpy as np, os

In [6]:
len(os.listdir("../../2023_covid/test_covid/unzip_ct/"))

4308

In [4]:
with open('/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/csv_pickle/filter_slice_train_dic1_05_challenge.pickle', 'rb') as f:
    train_dic = pickle.load(f)

with open('/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/csv_pickle/filter_slice_valid_dic1_05_challenge.pickle', 'rb') as f:
    valid_dic = pickle.load(f)

In [5]:
# del error axis [only train]
# negative/extra_challenge1_non_cov_train_new_icassp/ct_scan3897
# negative/extra_challenge1_non_cov_train_new_icassp/ct_scan3134
# negative/ct_scan_781 
# negative/train_non_covid/ct_scan1073
print(len(train_dic))
del train_dic["/ssd8/2023COVID19/Train_Valid_dataset/train_pure_crop/negative/ct_scan_781"]
del train_dic["/ssd8/2023COVID19/Train_Valid_dataset/train_pure_crop/negative/ct_scan1073"]
del train_dic["/ssd8/2023COVID19/Unzip_dataset/train/negative/extra_challenge1_non_cov_train_new_icassp/ct_scan3897"]
del train_dic["/ssd8/2023COVID19/Unzip_dataset/train/negative/extra_challenge1_non_cov_train_new_icassp/ct_scan3134"]
print(len(train_dic))

3028


KeyError: '/ssd8/2023COVID19/Train_Valid_dataset/train_pure_crop/negative/ct_scan_781'

In [4]:
replace_list = ["extra_challenge1_non_cov_train_new_icassp","extra_challenge1_non_cov_val_new_icassp","extra_challenge1_cov_val_new_icassp"]
copy_train_dic = {}
for keys_, value in train_dic.items():
    if "extra_challenge1_non_cov_train_new_icassp" in keys_:
        old_path = "/ssd8/2023COVID19/Unzip_dataset/train/negative/"
        new_path = "/ssd8/2023COVID19/Unzip_dataset/train_pure_crop_challenge/negative/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_train_dic[new_key] = value
train_dic = copy_train_dic
copy_valid_dic = {}
for keys_, value in valid_dic.items():
    if "extra_challenge1_non_cov_val_new_icassp" in keys_:
        old_path = "/ssd8/2023COVID19/Unzip_dataset/valid/negative/"
        new_path = "/ssd8/2023COVID19/Unzip_dataset/valid_pure_crop_challenge/negative/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_valid_dic[new_key] = value
valid_dic = copy_valid_dic
copy_valid_dic = {}
for keys_, value in valid_dic.items():
    if "extra_challenge1_cov_val_new_icassp" in keys_:
        old_path = "/ssd8/2023COVID19/Unzip_dataset/valid/positive/"
        new_path = "/ssd8/2023COVID19/Unzip_dataset/valid_pure_crop_challenge/positive/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_valid_dic[new_key] = value
valid_dic = copy_valid_dic

In [5]:
train_df = pd.DataFrame(None, columns=['path', 'filename', 'label'])
valid_df = pd.DataFrame(None, columns=['path', 'filename', 'label'])

print("="*10, "loading data DataFrame", "="*10)
for path in list(train_dic.keys()):
    image_list = os.listdir(path)
    image_list.sort(key=lambda x: int(os.path.splitext(x)[0]))
    temp_df = pd.DataFrame([path]*len(image_list), columns=['path'])
    temp_df['filename'] = image_list
    if 'negative' in temp_df.path[0]:
        temp_df['label'] = [0]*len(image_list)
    else:
        temp_df['label'] = [1]*len(image_list)
    train_df = pd.concat([train_df, temp_df])

for path in list(valid_dic.keys()):
    image_list = os.listdir(path)
    image_list.sort(key=lambda x: int(os.path.splitext(x)[0]))
    temp_df = pd.DataFrame([path]*len(image_list), columns=['path'])
    temp_df['filename'] = image_list
    if 'negative' in temp_df.path[0]:
        temp_df['label'] = [0]*len(image_list)
    else:
        temp_df['label'] = [1]*len(image_list)
    valid_df = pd.concat([valid_df, temp_df]) 

========== loading data DataFrame ==========


In [6]:
import copy
cp_train_dic = copy.deepcopy(train_dic)
cp_valid_dic = copy.deepcopy(valid_dic)
print(len(train_dic), len(valid_dic))
print(train_df.shape, valid_df.shape)

3028 704
(539562, 3) (150886, 3)


In [7]:
# drop_count_t = 0
# filter_num = 5
# for path_ in train_dic:
#     i = len(os.listdir(path_))
#     if i <filter_num:
#         print(f"張數{i}", path_)
#         drop_count_t = drop_count_t + i
#         del cp_train_dic[path_]
#         train_df = train_df[train_df.path!=path_]
# drop_count_v = 0
# for path_ in valid_dic:
#     i = len(os.listdir(path_))
#     if i <filter_num:
#         print(f"張數{i}", path_)
#         drop_count_v = drop_count_v + i
#         del cp_valid_dic[path_]
#         valid_df = valid_df[valid_df.path!=path_]

# print(len(cp_train_dic), len(cp_valid_dic))
# print(train_df.shape, valid_df.shape)
# print(drop_count_t, drop_count_v)

In [8]:
train_df = train_df[['path', 'label']]
valid_df = valid_df[['path', 'label']]
train_df = train_df.drop_duplicates(subset='path')
valid_df = valid_df.drop_duplicates(subset='path')
print(train_df.shape, valid_df.shape)

(3028, 2) (704, 2)


In [10]:
with open('/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/filter_slice_train_dic1_05_challenge.pickle', 'wb') as handle:
    pickle.dump(cp_train_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/filter_slice_valid_dic1_05_challenge.pickle', 'wb') as handle:
    pickle.dump(cp_valid_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
train_df.to_csv("/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/filter_slice_train_df_challenge.csv", index=False, encoding='utf-8-sig')
valid_df.to_csv("/ssd8/2023COVID19/CT-COVID19-Classification/train_code_fix/filter_slice_valid_df_challenge.csv", index=False, encoding='utf-8-sig')